In [14]:
import os
current_directory = os.getcwd()
model_directory = os.path.join(current_directory, "model")
print(model_directory)

c:\Users\lch85\OneDrive\바탕 화면\ai\model


In [15]:
from datasets import load_dataset

dataset = load_dataset("CShorten/ML-ArXiv-Papers")["train"]

# Extract abstracts to train on and corresponding titles
abstracts = dataset["abstract"]
titles = dataset["title"]

Found cached dataset csv (C:/Users/lch85/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 199.33it/s]


In [45]:
len(abstracts)

117592

문장 분할

In [17]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
sentences = [sent_tokenize(abstract) for abstract in abstracts]
sentences = [sentence for doc in sentences for sentence in doc]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lch85\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


임베딩 모델

In [18]:
from sentence_transformers import SentenceTransformer

import torch
if torch.cuda.is_available():
    print("Using GPU")
else:
    print("Using CPU")

# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(abstracts, show_progress_bar=True)

Using GPU


Batches: 100%|██████████| 3675/3675 [00:59<00:00, 62.10it/s] 


차원 축소

In [19]:
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

토픽 수 조절

In [20]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

중지어 제거, 빈도 적은 단어 제거, n-그램: 한 두어개

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

다각적 주제 모델링

In [22]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)


# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model,
    "POS": pos_model
}

In [23]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

topics, probs = topic_model.fit_transform(abstracts, embeddings)

2023-10-13 18:05:32,860 - BERTopic - Reduced dimensionality
2023-10-13 18:05:37,800 - BERTopic - Clustered reduced embeddings


In [24]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,POS,Representative_Docs
0,-1,38819,-1_data_learning_model_models,"[data, learning, model, models, based, trainin...","[deep learning, classification, machine learni...","[data, learning, model, models, based, trainin...","[data, learning, model, models, training, perf...",[ There exist many high-dimensional data in r...
1,0,5550,0_policy_reinforcement_reinforcement learning_rl,"[policy, reinforcement, reinforcement learning...","[policy gradient, reinforcement learning, deep...","[policy, reinforcement, reinforcement learning...","[policy, reinforcement, learning, reward, agen...",[ The history of learning for control has bee...
2,1,3514,1_speech_audio_speaker_music,"[speech, audio, speaker, music, acoustic, asr,...","[automatic speech, speech enhancement, speech ...","[speech, audio, speaker, music, acoustic, asr,...","[speech, audio, speaker, music, acoustic, reco...",[ Despite the rapid progress of automatic spe...
3,2,3275,2_3d_object_video_segmentation,"[3d, object, video, segmentation, image, objec...","[3d, point cloud, point clouds, object detecti...","[3d, object, video, segmentation, image, objec...","[object, video, segmentation, image, objects, ...",[ 3D object representation learning is a fund...
4,3,2111,3_graph_node_graphs_gnns,"[graph, node, graphs, gnns, nodes, graph neura...","[graph neural, graph classification, graph con...","[graph, node, graphs, gnns, nodes, graph neura...","[graph, node, graphs, nodes, networks, embeddi...",[ Message passing Graph Neural Networks (GNNs...
...,...,...,...,...,...,...,...,...
114,113,176,113_particle_physics_detector_lhc,"[particle, physics, detector, lhc, jet, collid...","[hadron collider, lhc, deep learning, particle...","[particle, physics, detector, lhc, jet, collid...","[particle, physics, detector, jet, collider, h...",[ Autoencoders have useful applications in hi...
115,114,158,114_clustering_deep clustering_deep_cluster,"[clustering, deep clustering, deep, cluster, u...","[deep clustering, deep learning, deep neural, ...","[clustering, deep clustering, deep, cluster, u...","[clustering, deep clustering, deep, cluster, u...","[ Recently, deep clustering, which is able to..."
116,115,156,115_attacks_attack_adversarial_rl,"[attacks, attack, adversarial, rl, drl, reinfo...","[adversarial training, adversarial attacks, ad...","[attacks, attack, adversarial, rl, drl, reinfo...","[attacks, attack, adversarial, reinforcement, ...",[ Due to the broad range of applications of r...
117,116,156,116_view_multi view_views_multi,"[view, multi view, views, multi, clustering, v...","[view clustering, view learning, view classifi...","[view, multi view, views, multi, clustering, v...","[view, views, multi, clustering, view clusteri...",[ Multi-view clustering has attracted much at...


In [36]:
a = topic_model.get_topic(-1, full=True)
print(a)

{'Main': [('data', 0.005508243175752185), ('learning', 0.005480041773925623), ('model', 0.0050207325673433155), ('models', 0.0045481094894343176), ('based', 0.0045077131217311555), ('training', 0.00391511730864696), ('performance', 0.003808629460358503), ('neural', 0.0037996598115721735), ('using', 0.003795561744545858), ('method', 0.003732200800767053)], 'KeyBERT': [('deep learning', 0.5497353), ('classification', 0.4077804), ('machine learning', 0.37792593), ('learning', 0.37790638), ('neural networks', 0.37687403), ('neural network', 0.35205984), ('neural', 0.34395343), ('datasets', 0.33463377), ('dataset', 0.33153087), ('deep', 0.33058673)], 'MMR': [('data', 0.005508243175752185), ('learning', 0.005480041773925623), ('model', 0.0050207325673433155), ('models', 0.0045481094894343176), ('based', 0.0045077131217311555), ('training', 0.00391511730864696), ('performance', 0.003808629460358503), ('neural', 0.0037996598115721735), ('using', 0.003795561744545858), ('method', 0.003732200800

In [43]:
b = a.get('KeyBERT')
temp = [name[0] for name in b]
temp[:5]

['deep learning',
 'classification',
 'machine learning',
 'learning',
 'neural networks']

라벨 커스텀마이징

In [26]:
# # Label the topics yourself
# topic_model.set_topic_labels({1: "Space Travel", 7: "Religion"})

# # or use one of the other topic representations, like KeyBERTInspired
# keybert_topic_labels = {topic: " | ".join(list(zip(*values))[0][:3]) for topic, values in topic_model.topic_aspects_["KeyBERT"].items()}
# topic_model.set_topic_labels(keybert_topic_labels)


In [27]:
# or use one of the other topic representations, like KeyBERTInspired
keybert_topic_labels = {topic: " | ".join(list(zip(*values))[0][:3]) for topic, values in topic_model.topic_aspects_["KeyBERT"].items()}
topic_model.set_topic_labels(keybert_topic_labels)

In [28]:
topic_model.get_topic_info()

,Topic,Count,Name,CustomName,Representation,KeyBERT,MMR,POS,Representative_Docs
0,-1,38819,-1_data_learning_model_models,deep learning | classification | machine learning,"[data, learning, model, models, based, trainin...","[deep learning, classification, machine learni...","[data, learning, model, models, based, trainin...","[data, learning, model, models, training, perf...",[ There exist many high-dimensional data in r...
1,0,5550,0_policy_reinforcement_reinforcement learning_rl,policy gradient | reinforcement learning | dee...,"[policy, reinforcement, reinforcement learning...","[policy gradient, reinforcement learning, deep...","[policy, reinforcement, reinforcement learning...","[policy, reinforcement, learning, reward, agen...",[ The history of learning for control has bee...
2,1,3514,1_speech_audio_speaker_music,automatic speech | speech enhancement | speech...,"[speech, audio, speaker, music, acoustic, asr,...","[automatic speech, speech enhancement, speech ...","[speech, audio, speaker, music, acoustic, asr,...","[speech, audio, speaker, music, acoustic, reco...",[ Despite the rapid progress of automatic spe...
3,2,3275,2_3d_object_video_segmentation,3d | point cloud | point clouds,"[3d, object, video, segmentation, image, objec...","[3d, point cloud, point clouds, object detecti...","[3d, object, video, segmentation, image, objec...","[object, video, segmentation, image, objects, ...",[ 3D object representation learning is a fund...
4,3,2111,3_graph_node_graphs_gnns,graph neural | graph classification | graph co...,"[graph, node, graphs, gnns, nodes, graph neura...","[graph neural, graph classification, graph con...","[graph, node, graphs, gnns, nodes, graph neura...","[graph, node, graphs, nodes, networks, embeddi...",[ Message passing Graph Neural Networks (GNNs...
...,...,...,...,...,...,...,...,...,...
114,113,176,113_particle_physics_detector_lhc,hadron collider | lhc | deep learning,"[particle, physics, detector, lhc, jet, collid...","[hadron collider, lhc, deep learning, particle...","[particle, physics, detector, lhc, jet, collid...","[particle, physics, detector, jet, collider, h...",[ Autoencoders have useful applications in hi...
115,114,158,114_clustering_deep clustering_deep_cluster,deep clustering | deep learning | deep neural,"[clustering, deep clustering, deep, cluster, u...","[deep clustering, deep learning, deep neural, ...","[clustering, deep clustering, deep, cluster, u...","[clustering, deep clustering, deep, cluster, u...","[ Recently, deep clustering, which is able to..."
116,115,156,115_attacks_attack_adversarial_rl,adversarial training | adversarial attacks | a...,"[attacks, attack, adversarial, rl, drl, reinfo...","[adversarial training, adversarial attacks, ad...","[attacks, attack, adversarial, rl, drl, reinfo...","[attacks, attack, adversarial, reinforcement, ...",[ Due to the broad range of applications of r...
117,116,156,116_view_multi view_views_multi,view clustering | view learning | view classif...,"[view, multi view, views, multi, clustering, v...","[view clustering, view learning, view classifi...","[view, multi view, views, multi, clustering, v...","[view, views, multi, clustering, view clusteri...",[ Multi-view clustering has attracted much at...


In [29]:
topic_distr, _ = topic_model.approximate_distribution(abstracts, window=8, stride=4)
# topic_distr, _ = topic_model.approximate_distribution(docs, use_embedding_model=True)

100%|██████████| 118/118 [01:01<00:00,  1.92it/s]


In [30]:
abstract_id = 10
print(abstracts[abstract_id])

  Speaker identification is a powerful, non-invasive and in-expensive biometric
technique. The recognition accuracy, however, deteriorates when noise levels
affect a specific band of frequency. In this paper, we present a sub-band based
speaker identification that intends to improve the live testing performance.
Each frequency sub-band is processed and classified independently. We also
compare the linear and non-linear merging techniques for the sub-bands
recognizer. Support vector machines and Gaussian Mixture models are the
non-linear merging techniques that are investigated. Results showed that the
sub-band based method used with linear merging techniques enormously improved
the performance of the speaker identification over the performance of wide-band
recognizers when tested live. A live testing improvement of 9.78% was achieved



In [31]:
topic_model.visualize_distribution(topic_distr[abstract_id])

In [32]:
topic_model.visualize_distribution(topic_distr[abstract_id], custom_labels=True)

In [33]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(abstracts[abstract_id], calculate_tokens=True)

# Visualize the token-level distributions
df = topic_model.visualize_approximate_distribution(abstracts[abstract_id], topic_token_distr[0])
df

100%|██████████| 1/1 [00:00<00:00, 10.42it/s]


,Speaker,identification,is,powerful,non,invasive,and,in,expensive,biometric,technique,The,recognition,accuracy,however,deteriorates,when,noise,levels,affect,specific,band,of,frequency,In,this,paper,we,present,sub,band,based,speaker,identification,that,intends,to,improve,the,live,testing,performance,Each,frequency,sub,band,is,processed,and,classified,independently,We,also,compare,the,linear,and,non,linear,merging,techniques,for,the,sub,bands,recognizer,Support,vector,machines,and,Gaussian,Mixture,models,are,the,non,linear,merging,techniques,that,are,investigated,Results,showed,that,the,sub,band,based,method,used,with,linear,merging,techniques,enormously,improved,the,performance,of,the,speaker,identification,over,the,performance,of,wide,band,recognizers,when,tested,live,live,testing,improvement,of,78,was,achieved
1_speech_audio_speaker_music,0.112,0.112,0.112,0.112,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.100,0.100,0.100,0.100,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.110,0.228,0.346,0.464,0.354,0.236,0.118,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
15_gaussian_variational_inference_gp,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.113,0.226,0.226,0.226,0.113,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
66_emotion_emotion recognition_recognition_facial,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.112,0.112,0.112,0.112,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
95_svm_support vector_support_vector,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.236,0.567,0.989,1.142,0.906,0.575,0.153,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
100_handwritten_text_character_characters,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.114,0.114,0.114,0.114,0.000,0.000,

In [34]:
# Reduce outliers
new_topics = topic_model.reduce_outliers(abstracts, topics)

# Reduce outliers with pre-calculate embeddings instead
new_topics = topic_model.reduce_outliers(abstracts, topics, strategy="embeddings", embeddings=embeddings)

 72%|███████▏  | 28/39 [00:30<00:12,  1.11s/it]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "c:\Users\lch85\miniconda3\envs\cap\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lch85\AppData\Local\Temp\ipykernel_12380\2026386901.py", line 2, in <module>
    new_topics = topic_model.reduce_outliers(abstracts, topics)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lch85\miniconda3\envs\cap\Lib\site-packages\bertopic\_bertopic.py", line 2109, in reduce_outliers
    topic_distr, _ = self.approximate_distribution(outlier_docs, min_similarity=threshold, **distributions_params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lch85\miniconda3\envs\cap\Lib\site-packages\bertopic\_bertopic.py", line -1, in approximate_distribution
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (mos

In [ ]:
topic_model.update_topics(docs, topics=new_topics)